# Authentication - Rest API - OAuth DeviceCode

## Set your variables here

In [3]:
<#
$config = @{
    tenantDomain = "domain.com"
    tenantID = "xxxxxxx-xxxx-xxxx-xxxxx-xxxxxxxxxx"          ## Azure AD Tenant ID
    appID = "xxxxxx-23af-4ae5-a6ed-xxxxxx"                   ## Usually known as Client ID or Application ID
    appSecret = "W_xxxxxxxxxxxxxxxxxx.xxxxxxxxx"             ## This is the secretKey. Important only authorized users have this 
    resourceURI = "https://graph.microsoft.com"   ## This is the resource endpoint for MDATP
}
#>

## Import credential from storage path or Export a new XML file using above parameters

In [1]:
## Checks to see if config file exist and if not export credential file from above.
$storagePath = ".\graph.credential"
$config = (Import-CliXml -Path $storagePath)
if (!$config) {
    $config | Export-CliXml -Path $storagePath
} else {
    Write-Host -Foregroundcolor green "`nCredential file loaded from $($storagePath)"
}

The below script needs to be able to find the current output cell; this is an easy method to get it.


Credential file loaded from .\graph.credential


## Verify if parameters look good

In [21]:
$authUrl = "https://login.microsoftonline.com/$($config.tenantDomain)";
Write-Host -ForeGroundColor Yellow "Authentication Parameters"
Write-Host "Resource URL: $($config.resourceURI)"
Write-Host "Client APP ID: $($config.AppId -replace "\d","*")"
Write-Host "AuthUrl: $($authUrl)"

Authentication Parameters
Resource URL: https://graph.microsoft.com/
Client APP ID: ae*fa***-**af-*ae*-a*ed-*ab*******c*
AuthUrl: https://login.microsoftonline.com/jingtoso.com


## Make Rest API Call and authenticate via Device Code

In [17]:
$postParams = @{ 
    resource = $config.resourceURI 
    client_id = $config.AppID
}
$tokenResponse = $null; 
$response = Invoke-RestMethod -Method POST -Uri "$authurl/oauth2/devicecode" -Body $postParams
$tokenParams = @{ grant_type = "device_code"; resource = $config.resourceURI; client_id = $config.AppID; code = "$($response.device_code)" }
Write-Host $response.message
#I got tired of manually copying the code, so I did string manipulation and stored the code in a variable and added to the clipboard automatically
$code = $response.message -match "code\s(.+)\sto"
$code = $Matches[1]
Set-Clipboard -Value $code

Write-Host "Waiting for code"
While (!$tokenResponse) {
    Try {
        $tokenResponse = Invoke-RestMethod -Method POST -Uri "$authurl/oauth2/token" -Body $tokenParams -ErrorAction Ignore
        Write-Host -ForeGroundColor Green "`nReceived Token!"
        Write-Host -ForegroundColor Green "Connected and Access Token received and will expire $($tokenResponse.expires_on)"
    } Catch {
    }
}

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code EHATMFJXG to authenticate.
Waiting for code

Received Token!
Connected and Access Token received and will expire 1592348346


## Check if code valid

In [32]:
$tokenResponse | Select-Object -Property @('token_type','scope','expires_on','resource') | FL


token_type : Bearer
scope      : EntitlementManagement.Read.All Reports.Read.All User.Read User.Read.All 
             User.ReadBasic.All
expires_on : 1592348346
resource   : https://graph.microsoft.com/




## Query API and filter time period to last '7' days. 

In [45]:
$howManyDays = 7   # How many days do you want to go back

## Request Parameters
$URI = "https://graph.microsoft.com/beta/reports/getEmailActivityCounts(period='D$($howManyDays)')?`$format=application/json"  ## URL of where to query in the graph API/beta/reports/getEmailActivityCounts(period='D7')?$format=text/csv

$authHeader = @{ 
    'Content-Type' = 'application/json'
    Accept = 'application/json'
    Authorization = "Bearer $($tokenResponse.access_token)" 
}

#query Graph API and insert into $graphresponse variable
$Result = (Invoke-RestMethod -Method Get -Uri $URI -Headers $authHeader -ErrorAction Stop).value
$Result | Select -First 5 | Format-Table


@odata.type                           reportRefreshDate send receive read reportDate reportPeriod
-----------                           ----------------- ---- ------- ---- ---------- ------------
#microsoft.graph.emailActivitySummary 2020-06-15           6      59      2020-06-15 7
#microsoft.graph.emailActivitySummary 2020-06-15           2      52 1    2020-06-14 7
#microsoft.graph.emailActivitySummary 2020-06-15           2      49      2020-06-13 7
#microsoft.graph.emailActivitySummary 2020-06-15           2      46      2020-06-12 7
#microsoft.graph.emailActivitySummary 2020-06-15           6      57      2020-06-11 7



## Clean up any empty counts in send and receive and replace with 0 and generate Scatter Report based off email Activity

In [44]:
ForEach ($item in $Result) {
    if ($item.send -eq $null) {
        $item.send = 0
    }
    if ($item.receive -eq $null) {
        $item.receive = 0
    }
}

$sendSeries = [Graph.Scatter]@{
    name = "Emails Sent"
    x = $Result.reportDate
    y = $Result.send
}

$receiveSeries = [Graph.Scatter]@{
    name = "Emails Received"
    x = $Result.reportDate
    y = $Result.receive
}

$chart = @($receiveSeries, $sendSeries) | New-PlotlyChart -Title "Emails Received vs Sent"
Out-Display $chart